<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий на hh.ru
   

In [1]:
clients = [8, 4, 9, 6, 7, 1, 6, 8, 9, 10]
def CSI_sort(clients):
    new_list = sorted(clients)
    max_grade = []
    for elem in new_list:
        if elem not in max_grade:
            max_grade.append(elem)
    return max_grade[-3:]
CSI_sort(clients)

[8, 9, 10]

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
#%matplotlib inline
import seaborn as sns

import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff

# Параметры общего стиля
#sns.set_style("whitegrid")
colors = sns.color_palette("pastel")
figsize = (6, 4)    # Для одного фрейма
figsize2 = (12, 4)  # Для двух фреймов
figsize3 = (16, 4)  # Для трёх фреймов

# Исследование структуры данных

1. Прочитайте данные с помощью библиотеки Pandas. Совет: перед чтением обратите внимание на разделитель внутри файла. 

In [3]:
#hh_data = pd.read_csv('/content/dst-3.0_16_1_hh_database.csv', sep=';')
hh_data = pd.read_csv('https://www.dropbox.com/s/k2qjietbvhdzqqs/dst-3.0_16_1_hh_database.csv?raw=true', sep=';')

2. Выведите несколько первых (последних) строк таблицы, чтобы убедиться в том, что ваши данные не повреждены. Ознакомьтесь с признаками и их структурой.

In [4]:
display(hh_data.head())

,"Пол, возраст",ЗП,Ищет работу на должность:,"Город, переезд, командировки",Занятость,График,Опыт работы,Последнее/нынешнее место работы,Последняя/нынешняя должность,Образование и ВУЗ,Обновление резюме,Авто
0,"Мужчина , 39 лет , родился 27 ноября 1979",29000 руб.,Системный администратор,"Советск (Калининградская область) , не готов к...","частичная занятость, проектная работа, полная ...","гибкий график, полный день, сменный график, ва...",Опыт работы 16 лет 10 месяцев Август 2010 — п...,"МАОУ ""СОШ № 1 г.Немана""",Системный администратор,Неоконченное высшее образование 2000 Балтийск...,16.04.2019 15:59,Имеется собственный автомобиль
1,"Мужчина , 60 лет , родился 20 марта 1959",40000 руб.,Технический писатель,"Королев , не готов к переезду , готов к редким...","частичная занятость, проектная работа, полная ...","гибкий график, полный день, сменный график, уд...",Опыт работы 19 лет 5 месяцев Январь 2000 — по...,Временный трудовой коллектив,"Менеджер проекта, Аналитик, Технический писатель",Высшее образование 1981 Военно-космическая ак...,12.04.2019 08:42,Не указано
2,"Женщина , 36 лет , родилась 12 августа 1982",20000 руб.,Оператор,"Тверь , не готова к переезду , не готова к ком...",полная занятость,полный день,Опыт работы 10 лет 3 месяца Октябрь 2004 — Де...,ПАО Сбербанк,Кассир-операционист,Среднее специальное образование 2002 Профессио...,16.04.2019 08:35,Не указано
3,"Мужчина , 38 лет , родился 25 июня 1980",100000 руб.,Веб-разработчик (HTML / CSS / JS / PHP / базы ...,"Саратов , не готов к переезду , готов к редким...","частичная занятость, проектная работа, полная ...","гибкий график, удаленная работа",Опыт работы 18 лет 9 месяцев Август 2017 — Ап...,OpenSoft,Инженер-программист,Высшее образование 2002 Саратовский государст...,08.04.2019 14:23,Не указано
4,"Женщина , 26 лет , родилась 3 марта 1993",140000 руб.,Региональный менеджер по продажам,"Москва , не готова к переезду , готова к коман...",полная занятость,полный день,Опыт работы 5 лет 7 месяцев Региональный мене...,Мармелад,Менеджер по продажам,Высшее образование 2015 Кгу Психологии и педаг...,22.04.2019 10:32,Не указано


3. Выведите основную информацию о числе непустых значений в столбцах и их типах в таблице.

4. Обратите внимание на информацию о числе непустых значений.

In [5]:
hh_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44744 entries, 0 to 44743
Data columns (total 12 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   Пол, возраст                     44744 non-null  object
 1   ЗП                               44744 non-null  object
 2   Ищет работу на должность:        44744 non-null  object
 3   Город, переезд, командировки     44744 non-null  object
 4   Занятость                        44744 non-null  object
 5   График                           44744 non-null  object
 6   Опыт работы                      44576 non-null  object
 7   Последнее/нынешнее место работы  44743 non-null  object
 8   Последняя/нынешняя должность     44742 non-null  object
 9   Образование и ВУЗ                44744 non-null  object
 10  Обновление резюме                44744 non-null  object
 11  Авто                             44744 non-null  object
dtypes: object(12)
memory usage: 4.1+

5. Выведите основную статистическую информацию о столбцах.


In [6]:
hh_data.describe()

,"Пол, возраст",ЗП,Ищет работу на должность:,"Город, переезд, командировки",Занятость,График,Опыт работы,Последнее/нынешнее место работы,Последняя/нынешняя должность,Образование и ВУЗ,Обновление резюме,Авто
count,44744,44744,44744,44744,44744,44744,44576,44743,44742,44744,44744,44744
unique,16003,690,14929,10063,38,47,44413,30214,16927,40148,18838,2
top,"Мужчина , 32 года , родился 17 сентября 1986",50000 руб.,Системный администратор,"Москва , не готов к переезду , не готов к кома...",полная занятость,полный день,Опыт работы 10 лет 8 месяцев Апрель 2018 — по...,Индивидуальное предпринимательство / частная п...,Системный администратор,Высшее образование 1987 Военный инженерный Кра...,07.05.2019 09:50,Не указано
freq,18,4064,3099,1261,30026,22727,3,935,2062,4,25,32268


# Преобразование данных

1. Начнем с простого - с признака **"Образование и ВУЗ"**. Его текущий формат это: **<Уровень образования год выпуска ВУЗ специальность...>**. Например:
* Высшее образование 2016 Московский авиационный институт (национальный исследовательский университет)...
* Неоконченное высшее образование 2000  Балтийская государственная академия рыбопромыслового флота…
Нас будет интересовать только уровень образования.

Создайте с помощью функции-преобразования новый признак **"Образование"**, который должен иметь 4 категории: "высшее", "неоконченное высшее", "среднее специальное" и "среднее".

Выполните преобразование, ответьте на контрольные вопросы и удалите признак "Образование и ВУЗ".

Совет: обратите внимание на структуру текста в столбце **"Образование и ВУЗ"**. Гарантируется, что текущий уровень образования соискателя всегда находится в первых 2ух слов и начинается с заглавной буквы. Воспользуйтесь этим.

*Совет: проверяйте полученные категории, например, с помощью метода unique()*


In [7]:
def get_education(arg):
    """[Функция преобразования признака Образование и ВУЗ]

    Args:
        args ([str]): [На вход получает сождержание признака 'Образование и ВУЗ']

    Returns:
        [str]: [Возвращает категорию образования]
    """
    arg = ' '.join(arg.split(' ')[:3])
    if 'Высшее' in arg:
        return 'высшее'
    elif 'Неоконченное высшее' in arg:
        return 'неоконченное высшее'
    elif 'Среднее специальное' in arg:
        return 'среднее специальное'
    elif 'Среднее образование' in arg:
        return 'среднее'

        
hh_data['Образование'] = hh_data['Образование и ВУЗ'].apply(get_education)
hh_data = hh_data.drop('Образование и ВУЗ', axis=1)

# Выделяем категориальные признаки
hh_data['Образование'] = hh_data['Образование'].astype('category')


hh_data['Образование'].value_counts(dropna=False)

высшее                 33863
среднее специальное     5765
неоконченное высшее     4557
среднее                  559
Name: Образование, dtype: int64

2. Теперь нас интересует столбец **"Пол, возраст"**. Сейчас он представлен в формате **<Пол , возраст , дата рождения >**. Например:
* Мужчина , 39 лет , родился 27 ноября 1979 
* Женщина , 21 год , родилась 13 января 2000
Как вы понимаете, нам необходимо выделить каждый параметр в отдельный столбец.

Создайте два новых признака **"Пол"** и **"Возраст"**. При этом важно учесть:
* Признак пола должен иметь 2 уникальных строковых значения: 'М' - мужчина, 'Ж' - женщина. 
* Признак возраста должен быть представлен целыми числами.

Выполните преобразование, ответьте на контрольные вопросы и удалите признак **"Пол, возраст"** из таблицы.

*Совет: обратите внимание на структуру текста в столбце, в части на то, как разделены параметры пола, возраста и даты рождения между собой - символом ' , '. 
Гарантируется, что структура одинакова для всех строк в таблице. Вы можете воспользоваться этим.*


In [8]:
def get_gender(args):
    """[Фнкция преобразавания признака Пол]

    Args:
        args ([str]): [Пол, возраст]

    Returns:
        [str]: [2 уникальных строковых значения: 'М' - мужчина, 'Ж' - женщина.]
    """
    if 'Мужчина ' in args:
        return 'м'
    if 'Женщина ' in args:
        return 'ж'


def get_age(args):
    """[Фнкция преобразавания признака ВОЗРАСТ]

    Args:
        args ([str]): [Пол, возраст]

    Returns:
        [str]: [Возраст]
    """
    splitted_args = args.split(',')
    age = splitted_args[1].split(' ')
    return age[2]

hh_data['Пол'] = hh_data['Пол, возраст'].apply(get_gender)

hh_data['Возраст'] = hh_data['Пол, возраст'].apply(get_age) 

# Преобразуем данные для оптимизации памяти
hh_data['Возраст'] = hh_data['Возраст'].astype(np.int8)  

percentage_gender = hh_data.groupby('Пол')['Пол'].count().transform(lambda x: x / hh_data.shape[0] * 100)
print('Распределение резюме %', round(percentage_gender, 2))
print('Средний возраст соискателей', round(hh_data['Возраст'].mean(), 1))

# Удаляем ненужный признак
hh_data = hh_data.drop('Пол, возраст', axis=1)

Распределение резюме % Пол
ж    19.07
м    80.93
Name: Пол, dtype: float64
Средний возраст соискателей 32.2


3. Следующим этапом преобразуем признак **"Опыт работы"**. Его текущий формат - это: **<Опыт работы: n лет m месяцев, периоды работы в различных компаниях…>**. 

Из столбца нам необходимо выделить общий опыт работы соискателя в месяцах, новый признак назовем "Опыт работы (месяц)"

Для начала обсудим условия решения задачи:
* Во-первых, в данном признаке есть пропуски. Условимся, что если мы встречаем пропуск, оставляем его как есть (функция-преобразование возвращает NaN)
* Во-вторых, в данном признаке есть скрытые пропуски. Для некоторых соискателей в столбце стоит значения "Не указано". Их тоже обозначим как NaN (функция-преобразование возвращает NaN)
* В-третьих, нас не интересует информация, которая описывается после указания опыта работы (периоды работы в различных компаниях)
* В-четвертых, у нас есть проблема: опыт работы может быть представлен только в годах или только месяцах. Например, можно встретить следующие варианты:
    * Опыт работы 3 года 2 месяца…
    * Опыт работы 4 года…
    * Опыт работы 11 месяцев…
    * Учитывайте эту особенность в вашем коде

Учитывайте эту особенность в вашем коде

В результате преобразования у вас должен получиться столбец, содержащий информацию о том, сколько месяцев проработал соискатель.
Выполните преобразование, ответьте на контрольные вопросы и удалите столбец **"Опыт работы"** из таблицы.


In [9]:
def get_experience(arg):
    """[Функция приведения отображения опыта работы к общему показателю в месяцах]

    Args:
        arg ([str]): [На вход получает значение признака 'Опыт работы']

    Returns:
        [int]: [Кол-во месяцев]
    """
    if arg == 'Не указано':
        return np.nan                               # В соответствии с заданием заполняем там где "не указано" NaN
    try:
        args_splited = arg.split(' ')[:6]
        year_key_words = ['лет', 'год', 'года']
        month_key_words = ['месяца', 'месяцев', 'месяц']
        year = 0
        month = 0
        for i in range(len(args_splited)):          # Перебираем переданные слова 
            if args_splited[i] in year_key_words:   # если слово совпадает с ключом
                year =  args_splited[i-1]           # Заносим предыдущее перед ним значение в переменную
            if args_splited[i] in month_key_words:
                month = args_splited[i-1]
        return round((int(year)*12) + (int(month)), 0)        
    except AttributeError:
        hh_data['Опыт работы'].fillna(np.nan)       # Если встречаем данные с которыми нельзя работать заполняем их NaN


hh_data['Опыт работы (месяц)'] = hh_data['Опыт работы'].apply(get_experience)
hh_data = hh_data.drop('Опыт работы', axis=1)

# Чему равен медианный опыт работы (в месяцах) в нашей таблице?
print('Медианный опыт работы (в месяцах) равен', round(hh_data['Опыт работы (месяц)'].median()))


Медианный опыт работы (в месяцах) равен 100


4. Хорошо идем! Следующий на очереди признак "Город, переезд, командировки". Информация в нем представлена в следующем виде: **<Город , (метро) , готовность к переезду (города для переезда) , готовность к командировкам>**. В скобках указаны необязательные параметры строки. Например, можно встретить следующие варианты:

* Москва , не готов к переезду , готов к командировкам
* Москва , м. Беломорская , не готов к переезду, не готов к командировкам
* Воронеж , готов к переезду (Сочи, Москва, Санкт-Петербург) , готов к командировкам

Создадим отдельные признаки **"Город"**, **"Готовность к переезду"**, **"Готовность к командировкам"**. При этом важно учесть:

* Признак **"Город"** должен содержать только 4 категории: "Москва", "Санкт-Петербург" и "город-миллионник" (их список ниже), остальные обозначьте как "другие".

    Список городов-миллионников:
    
   <code>million_cities = ['Новосибирск', 'Екатеринбург','Нижний Новгород','Казань', 'Челябинск','Омск', 'Самара', 'Ростов-на-Дону', 'Уфа', 'Красноярск', 'Пермь', 'Воронеж','Волгоград']
    </code>
    Инфорация о метро, рядом с которым проживает соискатель нас не интересует.
* Признак **"Готовность к переезду"** должен иметь два возможных варианта: True или False. Обратите внимание, что возможны несколько вариантов описания готовности к переезду в признаке "Город, переезд, командировки". Например:
    * … , готов к переезду , …
    * … , не готова к переезду , …
    * … , готова к переезду (Москва, Санкт-Петербург, Ростов-на-Дону)
    * … , хочу переехать (США) , …
    
    Нас интересует только сам факт возможности или желания переезда.
* Признак **"Готовность к командировкам"** должен иметь два возможных варианта: True или False. Обратите внимание, что возможны несколько вариантов описания готовности к командировкам в признаке "Город, переезд, командировки". Например:
    * … , готов к командировкам , … 
    * … , готова к редким командировкам , …
    * … , не готов к командировкам , …
    
    Нас интересует только сам факт готовности к командировке.
    
    Еще один важный факт: при выгрузки данных у некоторых соискателей "потерялась" информация о готовности к командировкам. Давайте по умолчанию будем считать, что такие соискатели не готовы к командировкам.
    
Выполните преобразования и удалите столбец **"Город, переезд, командировки"** из таблицы.

*Совет: обратите внимание на то, что структура текста может меняться в зависимости от указания ближайшего метро. Учите это, если будете использовать порядок слов в своей программе.*


In [10]:
# Список городов-миллионников:
million_cities = ['Новосибирск', 'Екатеринбург', 'Нижний Новгород', 'Казань', 'Челябинск', 'Омск', 'Самара', 'Ростов-на-Дону', 'Уфа', 'Красноярск', 'Пермь', 'Воронеж', 'Волгоград' ]


def get_citi(args):
    """[Функция определения категории города]

    Args:
        args ([str]): [Город, переезд, командировки]

    Returns:
        [str]: [Возвращает 4 категории: "Москва", "Санкт-Петербург" и "город-миллионник" и "другие" ]
    """
    args_splited = args.split(' ')[:1]
    if 'Москва' in args_splited:
        return 'Москва'
    if 'Санкт-Петербург' in args_splited:
        return 'Санкт-Петербург'
    for i in range(len(args_splited)):
        if args_splited[i] in million_cities:
            return 'город-миллионник'
    else:
        return 'другие'


hh_data['Город'] = hh_data['Город, переезд, командировки'].apply(get_citi)

# Сколько процентов соискателей живут в Санкт-Петербурге?
urban_residents = hh_data.groupby('Город')['Город'].count().transform(lambda x: x / hh_data.shape[0] * 100)
print(round(urban_residents[1]), '% соискателей живут в Санкт-Петербурге')


def ready_to_move(args):
    """[Функция формирования данных для признака Готовность к переезду]

    Args:
        args ([str]): [Город, переезд, командировки]

    Returns:
        [bool]: [готов = True, не готов = False]
    """
    result = None
    if ('не готов к переезду' in args) or ('не готова к переезду' in args) or ('не хочу переезжать' in args):
        result = False  
    else:
        result = True
    return result


hh_data['Готовность к переезду'] = hh_data['Город, переезд, командировки'].apply(ready_to_move)

# Сколько процентов соискателей готовы к переездам?
percent_to_move = hh_data.groupby('Готовность к переезду')['Готовность к переезду'].count().transform(lambda x: x / hh_data.shape[0] * 100)
print(round(percent_to_move[1]), '% соискателей готовы к переездам')


# «Готовность к командировкам». 
def ready_business_trip(args):
    """[Функция определения данных для признака Готовность к командировкам]

    Args:
        args ([str]): [Город, переезд, командировки]

    Returns:
        [bool]: [готов = True, не готов = False]
    """
    list_variants = ['готов к командировкам', 'готова к командировкам', 'готов к редким командировкам', 'готова к редким командировкам']
    args_splited = args.split(',')
    result = None
    for i in range(len(args_splited)):                 # Перебираем переданные слова 
        if args_splited[i].strip() in list_variants:   # если слово совпадает с ключом
            result = True
        else:
            result = False
    return result


hh_data['Готовность к командировкам'] = hh_data['Город, переезд, командировки'].apply(ready_business_trip)
hh_data = hh_data.drop('Город, переезд, командировки', axis=1)

# Выделяем категориальные признаки
hh_data['Город'] = hh_data['Город'].astype('category')

# Сколько процентов соискателей готовы одновременно и к переездам, и к командировкам?
percent_business_trip = round(hh_data[hh_data['Готовность к переезду'] & hh_data['Готовность к командировкам']].shape[0] / hh_data.shape[0] *100)
print(round(percent_business_trip), '% соискателей готовы одновременно и к переездам, и к командировкам')

11 % соискателей живут в Санкт-Петербурге
36 % соискателей готовы к переездам
32 % соискателей готовы одновременно и к переездам, и к командировкам


5. Рассмотрим поближе признаки **"Занятость"** и **"График"**. Сейчас признаки представляют собой набор категорий желаемой занятости (полная занятость, частичная занятость, проектная работа, волонтерство, стажировка) и желаемого графика работы (полный день, сменный график, гибкий график, удаленная работа, вахтовый метод).
На сайте hh.ru соискатель может указывать различные комбинации данных категорий, например:
* полная занятость, частичная занятость
* частичная занятость, проектная работа, волонтерство
* полный день, удаленная работа
* вахтовый метод, гибкий график, удаленная работа, полная занятость

Такой вариант признаков имеет множество различных комбинаций, а значит множество уникальных значений, что мешает анализу. Нужно это исправить!

Давайте создадим признаки-мигалки для каждой категории: если категория присутствует в списке желаемых соискателем, то в столбце на месте строки рассматриваемого соискателя ставится True, иначе - False.

Такой метод преобразования категориальных признаков называется One Hot Encoding и его схема представлена на рисунке ниже:
<img src=https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/ohe.jpg>
Выполните данное преобразование для признаков "Занятость" и "График", ответьте на контрольные вопросы, после чего удалите их из таблицы

In [11]:
employments = ['полная занятость', 'частичная занятость',
              'проектная работа', 'волонтерство', 'стажировка']
charts = ['полный день', 'сменный график', 
         'гибкий график', 'удаленная работа',
         'вахтовый метод']
         
for employment, chart in zip(employments, charts):
    hh_data[employment] = hh_data['Занятость'].apply(lambda x: employment in x)
    hh_data[chart] = hh_data['График'].apply(lambda x: chart in x)

# Удаляем ненужные столбцы
hh_data = hh_data.drop('Занятость', axis=1)
hh_data = hh_data.drop('График', axis=1)

# Сколько людей ищут проектную работу или волонтёрство (в обоих столбцах стоит True)?
print((hh_data[hh_data['проектная работа'] & hh_data['волонтерство']].shape[0]),'чел. ищут проектную работу или волонтёрство')

# Сколько людей хотят работать вахтовым методом или с гибким графиком (в обоих столбцах стоит True)?
print((hh_data[hh_data['вахтовый метод'] & hh_data['гибкий график']].shape[0]), 'чел. хотят работать вахтовым методом или с гибким графиком')

436 чел. ищут проектную работу или волонтёрство
2311 чел. хотят работать вахтовым методом или с гибким графиком


6. (2 балла) Наконец, мы добрались до самого главного и самого важного - признака заработной платы **"ЗП"**. 
В чем наша беда? В том, что помимо желаемой заработной платы соискатель указывает валюту, в которой он бы хотел ее получать, например:
* 30000 руб.
* 50000 грн.
* 550 USD

Нам бы хотелось видеть заработную плату в единой валюте, например, в рублях. Возникает вопрос, а где взять курс валют по отношению к рублю?

На самом деле язык Python имеет в арсенале огромное количество возможностей получения данной информации, от обращения к API Центробанка, до использования специальных библиотек, например pycbrf. Однако, это не тема нашего проекта.

Поэтому мы пойдем в лоб: обратимся к специальным интернет-ресурсам для получения данных о курсе в виде текстовых файлов. Например, MDF.RU, данный ресурс позволяет удобно экспортировать данные о курсах различных валют и акций за указанные периоды в виде csv файлов. Мы уже сделали выгрузку курсов валют, которые встречаются в наших данных за период с 29.12.2017 по 05.12.2019. Скачать ее вы можете **на платформе**

Создайте новый DataFrame из полученного файла. В полученной таблице нас будут интересовать столбцы:
* "currency" - наименование валюты в ISO кодировке,
* "date" - дата, 
* "proportion" - пропорция, 
* "close" - цена закрытия (последний зафиксированный курс валюты на указанный день).


Перед вами таблица соответствия наименований иностранных валют в наших данных и их общепринятых сокращений, которые представлены в нашем файле с курсами валют. Пропорция - это число, за сколько единиц валюты указан курс в таблице с курсами. Например, для казахстанского тенге курс на 20.08.2019 составляет 17.197 руб. за 100 тенге, тогда итоговый курс равен - 17.197 / 100 = 0.17197 руб за 1 тенге.
Воспользуйтесь этой информацией в ваших преобразованиях.

<img src=https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/table.jpg>


Осталось только понять, откуда брать дату, по которой определяется курс? А вот же она - в признаке **"Обновление резюме"**, в нем содержится дата и время, когда соискатель выложил текущий вариант своего резюме. Нас интересует только дата, по ней бы и будем сопоставлять курсы валют.

Теперь у нас есть вся необходимая информация для того, чтобы создать признак "ЗП (руб)" - заработная плата в рублях.

После ответа на контрольные вопросы удалите исходный столбец заработной платы "ЗП" и все промежуточные столбцы, если вы их создавали.

Итак, давайте обсудим возможный алгоритм преобразования: 
1. Перевести признак "Обновление резюме" из таблицы с резюме в формат datetime и достать из него дату. В тот же формат привести признак "date" из таблицы с валютами.
2. Выделить из столбца "ЗП" сумму желаемой заработной платы и наименование валюты, в которой она исчисляется. Наименование валюты перевести в стандарт ISO согласно с таблицей выше.
3. Присоединить к таблице с резюме таблицу с курсами по столбцам с датой и названием валюты (подумайте, какой тип объединения надо выбрать, чтобы в таблице с резюме сохранились данные о заработной плате, изначально представленной в рублях). Значение close для рубля заполнить единицей 1 (курс рубля самого к себе)
4. Умножить сумму желаемой заработной платы на присоединенный курс валюты (close) и разделить на пропорцию (обратите внимание на пропуски после объединения в этих столбцах), результат занести в новый столбец "ЗП (руб)".


In [ ]:
# Перевести признак «Обновление резюме» из таблицы с резюме в формат datetime и достать из него дату
hh_data['Обновление резюме'] = pd.to_datetime(hh_data['Обновление резюме'])
hh_data['Дата'] = hh_data['Обновление резюме'].dt.date
hh_data['Дата'] = pd.to_datetime(hh_data['Дата'])

# Формирование датасета с курсами валют
df_currency = pd.read_csv('https://www.dropbox.com/s/kik91sgee5jhiz6/ExchangeRates.csv?raw=true', sep=',')

# Преобразовываем в формат datetime
df_currency['date'] = pd.to_datetime(df_currency['date'])              


def get_currency(args):
    """Функция определения наименования валюты в которой зарплата

    Args:
        args ([str]): [сумма и валюта в которой зарплата]

    Returns:
        [str]: [абревиатуру валюты]
    """    
    args_splited = args.split(' ')
    currency_salary = None
    for i in range(len(args_splited)):
        if args_splited[i].isalpha():
            currency_salary = args_splited[i]
        return args_splited[1]


# Создаем признак
hh_data['Валюта'] = hh_data['ЗП'].apply(get_currency)

# Cловарь валют для замены
currency_dict = {
    'грн.':	'UAH',
    'USD':	'USD',
    'EUR':	'EUR',
    'бел.руб.':	'BYN',
    'KGS':	'KGS',
    'сум':	'UZS',
    'AZN':	'AZN',
    'KZT':	'KZT',
    'руб.':	'RUB'
}

# Функция приведение наименования валюты к стандарту ISO
hh_data['Валюта'] = hh_data['Валюта'].apply(lambda x: currency_dict[x])


def get_salary(args):
    """Функция отсечения наименования валюты от суммы

    Args:
        args ([str]): [сумма и абревиатура валюты]

    Returns:
        [int]: [сумма зарплаты]
    """
    args_splited = args.split(' ')
    for i in range(len(args_splited)):
        if args_splited[i].isdigit():
            salary = int(args_splited[i])
        return salary


hh_data['ЗП'] = hh_data['ЗП'].apply(get_salary)


def rate_in_date(currency, data):
    """Функция определения курса валюты в запрашиваемый день

    Args:
        currency ([str]): [код валюты ISO]
        data ([datetime]): [дата на которую нужно определить котировку]

    Returns:
        [float]: [курс валюты]
    """
    currency_rate = None
    currency_with_proportion = ['KZT', 'KGS', 'UAH', 'UZS']     # Список валют в которых надо учитывать пропорцию курса
    currency_without_proportion = ['USD', 'EUR', 'BYN', 'AZN']  # Список валют где нет необходимости делить на пропорцию
    mask_date = (df_currency['date'] == data)                   # Маска для фильтрации по дате
    mask_cur = (df_currency['currency'] == currency)            # Маска фильтрации по валюте
    
    if currency == 'RUB':
        currency_rate = 1
    
    if currency in currency_with_proportion:
        currency_rate = float(df_currency[(mask_date)&(mask_cur)]['close']) \
                / int(df_currency[(mask_date)&(mask_cur)]['proportion'])
    
    if currency in currency_without_proportion:
        currency_rate = float(df_currency[(mask_date)&(mask_cur)]['close'])
    
    return currency_rate


def get_salary_rub(args):
    """Приведение зарплаты к рублю по курсу банка на дату публикации

    Args:
        args ([str]): [description]

    Returns:
        [int]: [Сумма в рублях по курсу нацбанка]
    """
    sum = int(args[0]) 
    data = args[1] 
    currency = args[2]
    rate = rate_in_date(currency, data)
    
    if currency == 'RUB':
        salary = sum
    else:
        salary = sum*rate # Если валюта не рубль умножаем на курс

    return salary
    
    
hh_data['ЗП (руб)'] = hh_data[['ЗП', 'Дата', 'Валюта']].apply(get_salary_rub, axis=1)
hh_data = hh_data.drop('ЗП', axis=1)
hh_data = hh_data.drop('Валюта', axis=1)
hh_data = hh_data.drop('Обновление резюме', axis=1)

# Чему равна желаемая медианная заработная плата соискателей в нашей таблице (в рублях)? В ответе укажите целое число тысяч.
print(f"Медианная заработная плата соискателей равна {hh_data['ЗП (руб)'].median()} руб")

# Исследование зависимостей в данных

1. Постройте распределение признака **"Возраст"**. Опишите распределение, отвечая на следующие вопросы: чему равна мода распределения, каковы предельные значения признака, в каком примерном интервале находится возраст большинства соискателей? Есть ли аномалии для признака возраста, какие значения вы бы причислили к их числу?
*Совет: постройте гистограмму и коробчатую диаграмму рядом.*

In [ ]:
fig = plt.figure('Распределение признака "Возраст"', figsize=figsize3)
fig.subplots_adjust(top=0.8)
fig.suptitle('Распределение признака "Возраст"', fontsize=18)

axes = fig.subplots(1, 2)
ax = axes[0]
ax1 = axes[1]

sns.boxplot(
    ax=ax,
    data=hh_data,
    x='Возраст', 
    orient='h',                   # Горизонтальная расположение
    palette=[colors[0], colors[1]]
)
ax.set_title('Распределение по возрасту', fontsize=14)

# Гистограмма
ages_of_clients = hh_data.groupby('Возраст')['Возраст'].count() # Сортируем и считаем каждую группу по возрасту

sns.histplot(
    ax=ax1,
    data=ages_of_clients,
    x="Возраст"          
)

ax1.set_title('Возрастные группы', fontsize=14)
ax1.set_xlabel('Возраст', fontsize=12)
ax1.set_ylabel('Кол-во чел.', fontsize=12)

plt.show()

In [ ]:
ages_of_clients.info()

Выводы по графику 
----
Мода распределения равна 30 годам.
Средний возраст соискателей 32 года

Предельные значения признака 14 и 100 лет.

Чуть больше половины соискателей находится в интервале примерно между 25 и 35 лет.   
86% соискателей в возрасте до 40 лет

На графике наблюдаются отклонения после 55 лет. В этой возрастной группе  1,42% соискателей.

Делать выводы о чистоте данных считаю преждевременным, так как не было иследования медианы опыта работы великовозрастных с соедней группой до 55 лет. 
Если среднее значение опыта будет превышает соседей, тогда это означает, что пенсионеры в поисках работы, но если же опыт ниже ближайших соседей из группы до 55 лет, тогда данные следует удалить воизбежания искажений в анализе


2. Постройте распределение признака **"Опыт работы (месяц)"**. Опишите данное распределение, отвечая на следующие вопросы: чему равна мода распределения, каковы предельные значения признака, в каком примерном интервале находится опыт работы большинства соискателей? Есть ли аномалии для признака возраста, какие значения вы бы причислили к их числу?
*Совет: постройте гистограмму и коробчатую диаграмму рядом.*

In [ ]:
fig = plt.figure('Распределение признака "Опыт работы (месяц)"', figsize=figsize3)
fig.subplots_adjust(top=0.8)
fig.suptitle('Распределение признака "Опыт работы (месяц)"', fontsize=18)

axes = fig.subplots(1, 2)
ax = axes[0]
ax1 = axes[1]

# Усы
sns.boxplot(
    ax=ax,
    data=hh_data,
    x='Опыт работы (месяц)', 
    orient='h',                   # Горизонтальная расположение
    palette=[colors[0], colors[1]]
)

ax.set_xlabel('Опыт в месяцах', fontsize=12)

# Гистограмма
expirience_of_clients = hh_data.groupby('Опыт работы (месяц)')['Опыт работы (месяц)'].count() # Сортируем и считаем каждую группу по возрасту

sns.histplot(
    ax=ax1,
    data=expirience_of_clients,
    x="Опыт работы (месяц)"          
)

#ax1.set_title('set_title', fontsize=14)
ax1.set_xlabel('Опыт в месяцах', fontsize=12)
ax1.set_ylabel('Кол-во чел.', fontsize=12)

plt.show()

# Чему равен максимальный опыт работы (в месяцах)?



Выводы
---
Мода распределения равна 81 мес.

Предельные значения признака 1 и 1188 мес.

Опыт работы большинства соискателей находится в интервале между 50 и 150 мес. (53%)
Среднее значение 114 мес.
 
Аномалия 1188 мес, но считаю, что прежде чем причислить их к грязным данным, нужно дополнительно исследовать возраст данной группы. Если ему 140, а он бодр и молод, тогда хвалим и всё равно удалить дабы не путал логику программы.

3. Постройте распределение признака **"ЗП (руб)"**. Опишите данное распределение, отвечая на следующие вопросы: каковы предельные значения признака, в каком примерном интервале находится заработная плата большинства соискателей? Есть ли аномалии для признака возраста? Обратите внимание на гигантские размеры желаемой заработной платы.
*Совет: постройте гистограмму и коробчатую диаграмму рядом.*


In [ ]:
fig = plt.figure('Распределение признака "ЗП (руб)"', figsize=figsize3)
fig.subplots_adjust(top=0.8)
fig.suptitle('Распределение признака "ЗП (руб)"', fontsize=18)

axes = fig.subplots(1, 2)
ax = axes[0]
ax1 = axes[1]

# Усы
sns.boxplot(
    ax=ax,
    data=hh_data,
    x='ЗП (руб)', 
    orient='h',                   # Горизонтальная расположение
    palette=[colors[0], colors[1]]
)

ax.set_xticklabels([f"{round(x)}млн." for x in ax.get_xticks()/1000000])

# Гистограмма
salary_of_clients = hh_data.groupby(['ЗП (руб)', 'Возраст'])['ЗП (руб)'].count()# Сортируем и считаем каждую группу по возрасту

sns.histplot(
    ax=ax1,
    data=salary_of_clients,
    y='Возраст',
    x="ЗП (руб)"          
)

ax1.set_ylabel('Возраст (лет)', fontsize=12)
ax1.set_xticklabels([f"{round(x)}млн." for x in ax1.get_xticks()/1000000]) # округление по оси X

plt.show()


# Cколько соискателей требуют заработную плату выше 1 миллиона рублей.
above_million = hh_data[hh_data['ЗП (руб)'] > 1000000].reset_index().count()
above_million[1]

Выводы
---
Предельные значения признака 1 руб. и 24'304'876 руб.

Минимальное значение можно объяснить тем, что возможно, соискатели закончили обучение и готовы пойти на практику без должного уровня оплаты. 
Либо в целях маркетингового хода укали оплату существенно ниже рынка в надежде, что их резюме будет выигрывать в показе по отношению к остальным.

Заработная плата большинства соискателей находится в интервале 50'000 - 60'000 руб

Пять человек желают зарплату выше 1 млн руб.

Есть аномалия для признака "возраст" 100 лет

4. Постройте диаграмму, которая показывает зависимость **медианной** желаемой заработной платы (**"ЗП (руб)"**) от уровня образования (**"Образование"**). Используйте для диаграммы данные о резюме, где желаемая заработная плата меньше 1 млн рублей.
*Сделайте выводы по представленной диаграмме: для каких уровней образования наблюдаются наибольшие и наименьшие уровни желаемой заработной платы? Как вы считаете, важен ли признак уровня образования при прогнозировании заработной платы?*

In [ ]:
sorted_salary = hh_data[hh_data['ЗП (руб)'] < 1000000]
sorted_salary.groupby('Образование')['ЗП (руб)'].median().plot()

Выводы
---
Высшее образование по медианному показателю оплачивается выше всех остальных категорий.

Наименьшие уровни желаемой заработной платы наблюдаются в категории среднее и среднеспециальное.

5. Постройте диаграмму, которая показывает распределение желаемой заработной платы (**"ЗП (руб)"**) в зависимости от города (**"Город"**). Используйте для диаграммы данные о резюме, где желая заработная плата меньше 1 млн рублей.
*Сделайте выводы по полученной диаграмме: как соотносятся медианные уровни желаемой заработной платы и их размах в городах? Как вы считаете, важен ли признак города при прогнозировании заработной платы?*

In [ ]:
fig = px.box(
    sorted_salary,
    y='Город',
    x='ЗП (руб)',
    color='Город',
    title='Распределение желаемой заработной платы в зависимости от города',
    notched=True,
    points="all"
)
fig.show()

Выводы
---
Наибольший показатель желаемой заработной платы (~924 тысячи рублей)зафиксирован в категории городов "другие".

Медианные уровни желаемой заработной платы вышел в Москве и Санкт-Петербурге, и их размах в этих городах соответственно шире.

Считаете, что признак города при прогнозировании заработной платы важен.

6. Постройте **многоуровневую столбчатую диаграмму**, которая показывает зависимость медианной заработной платы (**"ЗП (руб)"**) от признаков **"Готовность к переезду"** и **"Готовность к командировкам"**. Проанализируйте график, сравнив уровень заработной платы в категориях.

In [ ]:
hh_data.groupby(['Готовность к переезду', 'Готовность к командировкам'])['ЗП (руб)'].median().plot()

Выводы
---
Медианная заработная плата соискателей, готовых и к переезду, и к командировкам равна 66 тыс.руб.

Наибольший уровень ожидаемой зарплаты демонстрирует категория "Готов к переезду и командировкам".

Наименьший уровень зарплаты наблюдается у категории "Не готов к переезду и командировкам".

Готовность соискателей к командировкам демонстрирует очевидные более высокие ожидания по зарплате.

7. Постройте сводную таблицу, иллюстрирующую зависимость **медианной** желаемой заработной платы от возраста (**"Возраст"**) и образования (**"Образование"**). На полученной сводной таблице постройте **тепловую карту**. Проанализируйте тепловую карту, сравнив показатели внутри групп.

In [ ]:
data_pivot_table = pd.pivot_table(hh_data,
                                  index='Образование',
                                  columns='Возраст',
                                  values='ЗП (руб)',
                                  aggfunc=np.median)

# Тепловая карта  
fig = px.imshow(data_pivot_table)
fig.show()


Выводы
---
Для категории образования "высшее"наблюдается самый быстрый карьерный рост (то есть интенсивность роста заработной платы наибольшая).

Чем ниже уровень образования, тем меньше динамика роста уровня заработной платы.

8. Постройте **диаграмму рассеяния**, показывающую зависимость опыта работы (**"Опыт работы (месяц)"**) от возраста (**"Возраст"**). Опыт работы переведите из месяцев в года, чтобы признаки были в едином масштабе. Постройте на графике дополнительно прямую, проходящую через точки (0, 0) и (100, 100). Данная прямая соответствует значениям, когда опыт работы равен возрасту человека. Точки, лежащие на этой прямой и выше нее - аномалии в наших данных (опыт работы больше либо равен возрасту соискателя)

In [ ]:
# Преобразовываем опыт месяцы в года
transformed_expirience = hh_data[['Возраст', 'Опыт работы (месяц)']]
transformed_expirience['Опыт работы (месяц)'] = transformed_expirience['Опыт работы (месяц)'].apply(lambda x: x/12)
transformed_expirience.rename(columns={'Опыт работы (месяц)': 'Опыт работы (лет)'}, inplace=True)                   # Приводим наименование столбца в соответствие содержанию

fig = px.scatter(
    transformed_expirience, 
    x='Возраст', 
    y='Опыт работы (лет)'
)

# построить прямую для выявления аномалии
fig.add_trace(go.Scatter(x=[0, 100], y=[0, 100],
                    mode='lines',
                    name='Возраст=Стаж'))

fig.show()

Выводы 
---
7 точек находятся на линии "опыт работы равен возрасту человека" либо выше её - это очевидные аномалии в данных.

**Дополнительные баллы**

Для получения 2 дополнительных баллов по разведывательному анализу постройте еще два любых содержательных графика или диаграммы, которые помогут проиллюстрировать влияние признаков/взаимосвязь между признаками/распределения признаков. Приведите выводы по ним. Желательно, чтобы в анализе участвовали признаки, которые мы создавали ранее в разделе "Преобразование данных".


In [ ]:
# ваш код здесь
fig = px.scatter(
    transformed_expirience, 
    x='Возраст', 
    y='Опыт работы (лет)'
)

# построить прямую для выявления аномалии
fig.add_trace(go.Scatter(x=[10, 100], y=[0, 90],
                    mode='lines',
                    name='Возраст=Стаж'))

fig.show()

Выводы
---
Считаю, что линию выявления аномалий необходимо понизить, так как 

# Очистка данных

1. Начнем с дубликатов в наших данных. Найдите **полные дубликаты** в таблице с резюме и удалите их. 

In [ ]:
# пропуски
duplicates = hh_data[hh_data.duplicated()]
print('Число дубликтов: {}'.format(duplicates.shape[0]))
hh_data = hh_data.drop_duplicates()
print('В результате осталось после очистки записей: {}'.format(hh_data.shape[0]))

2. Займемся пропусками. Выведите информацию **о числе пропусков** в столбцах. 

In [ ]:
# выводим результирующую долю пропусков
cols_nan_percent = hh_data.isna().sum()
cols_with_nan = cols_nan_percent[cols_nan_percent > 0].sort_values(ascending=False)
cols_with_nan
#hh_data[hh_data['Опыт работы (месяц)'].isna() == True].reset_index()
#null_data = hh_data.isnull().sum()
#display(null_data[null_data > 0])

3. Итак, у нас есть пропуски в 3ех столбцах: **"Опыт работы (месяц)"**, **"Последнее/нынешнее место работы"**, **"Последняя/нынешняя должность"**. Поступим следующим образом: удалите строки, где есть пропуск в столбцах с местом работы и должностью. Пропуски в столбце с опытом работы заполните **медианным** значением.

In [ ]:
# drop(inplace=True)
# fillna(median_value)
# Создаю копию данных
hh_data_copy = hh_data.copy()

# Удаляю строчки с NaN из столбцов "Последнее/нынешнее место работы" и "Последняя/нынешняя должность"
hh_data_copy = hh_data_copy[~((hh_data_copy['Последнее/нынешнее место работы'].isna()) | 
                  (hh_data_copy['Последняя/нынешняя должность'].isna())
                  )]

print(f'Найдено {hh_data_copy["Опыт работы (месяц)"].isna().sum()} NAN значений')
# Заполняю NaN медианными значениями
hh_data_copy = hh_data_copy.fillna(value={'Опыт работы (месяц)': hh_data_copy['Опыт работы (месяц)'].median()})
print(f'Осталось {hh_data_copy["Опыт работы (месяц)"].isna().sum()} NAN значений.')
print('Результирующее среднее значение Опыт работы (месяц)', round(hh_data_copy['Опыт работы (месяц)'].mean(),0))

# Заменяю данных на очищенные
hh_data = hh_data_copy.copy()

4. Мы добрались до ликвидации выбросов. Сначала очистим данные вручную. Удалите резюме, в которых указана заработная плата либо выше 1 млн. рублей, либо ниже 1 тыс. рублей.

In [ ]:
# Сколько выбросов
# Cколько соискателей ищут заработную плату выше 1 миллиона рублей.
above_million = hh_data[hh_data['ЗП (руб)'] > 1000000].reset_index().count()
print(f'Зарплата свыше 1 млн руб. заявлена у {above_million[1]} чел.')

# Cколько соискателей указали желаемую зарплату ниже 1 тыс.рублей.
below_thousand = hh_data[hh_data['ЗП (руб)'] < 1000].reset_index().count()
print(f'Зарплата ниже 1 тыс.руб. заявлена у {below_thousand[1]} чел.')

# Удалите резюме, в которых указана заработная плата либо выше 1 млн
hh_data = hh_data[~(hh_data['ЗП (руб)'] > 1000000)]

# Удалите резюме, в которых указана заработная плата ниже 1 тыс. рублей
hh_data = hh_data[~(hh_data['ЗП (руб)'] < 1000)]

5. В процессе разведывательного анализа мы обнаружили резюме, в которых **опыт работы в годах превышал возраст соискателя**. Найдите такие резюме и удалите их из данных


In [ ]:
# Cколько соискателей имеют опыт превышающий их возраст
experience_exceeds_age = hh_data[(hh_data['Опыт работы (месяц)']/12) > hh_data['Возраст']].reset_index().count()
print(f'Опыт превышает возраст у {experience_exceeds_age[1]} чел.')

# Удалите резюме, в которых опыт превышает возраст
hh_data = hh_data[~((hh_data['Опыт работы (месяц)']/12) > hh_data['Возраст'])]
                    

6. В результате анализа мы обнаружили потенциальные выбросы в признаке **"Возраст"**. Это оказались резюме людей чересчур преклонного возраста для поиска работы. Попробуйте построить распределение признака в **логарифмическом масштабе**. Добавьте к графику линии, отображающие **среднее и границы интервала метода трех сигм**. Напомним, сделать это можно с помощью метода axvline. Например, для построение линии среднего будет иметь вид:

`histplot.axvline(log_age.mean(), color='k', lw=2)`

В какую сторону асимметрично логарифмическое распределение? Напишите об этом в комментарии к графику.
Найдите выбросы с помощью метода z-отклонения и удалите их из данных, используйте логарифмический масштаб. Давайте сделаем послабление на **1 сигму** (возьмите 4 сигмы) в **правую сторону**.

Выведите таблицу с полученными выбросами и оцените, с каким возрастом соискатели попадают под категорию выбросов?

In [ ]:
#  построить распределение признака в логарифмическом масштабе
fig, ax = plt.subplots(1, 1, figsize=(8, 4))
log_age = np.log(hh_data['Возраст'] + 1)
histplot = sns.histplot(log_age, bins=30, ax=ax)
histplot.axvline(log_age.mean(), color='k', lw=2)
histplot.axvline(log_age.mean()+4*log_age.std(), color='k', ls='--', lw=2)
histplot.axvline(log_age.mean()-3*log_age.std(), color='k', ls='--', lw=2)
histplot.set_title('логарифмическое распределение');

plt.show()

# Найдите выбросы с помощью метода z-отклонения
def outliers_z_score(data, feature, left=3, right=3, log_scale=False):
    if log_scale:
        x = np.log(data[feature]+1)
    else:
        x = data[feature]
    mu = x.mean()
    sigma = x.std()
    lower_bound = mu - left * sigma
    upper_bound = mu + right * sigma
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x > lower_bound) & (x < upper_bound)]
    return outliers, cleaned


outliers, cleaned = outliers_z_score(hh_data, 'Возраст', left=3,  right=4, log_scale=False)
print(f'Число выбросов по методу z-отклонения: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')

ваш коммментарий здесь